## Get zip files of building records using cesmd fetcher

In [1]:
import search
from pathlib import Path

In [2]:
BUILDINGS = {
    "CE24602": "Los Angeles - 52-story Office Bldg"
}

building_dir = Path("./buildings")
if not building_dir.exists():
    Path.mkdir(building_dir)

In [3]:
update_files = False

In [4]:
if update_files:
    for cesmd in BUILDINGS.keys():
        print(BUILDINGS[cesmd])
        search.get_records(str(building_dir)+"/"+cesmd, "cchern@berkeley.edu", station_code=cesmd, process_level="processed", include_inactive=True)

## Move records to top levels of unzipped folders

In [5]:
import glob
from zipfile import ZipFile

In [6]:
extract_new = False

In [7]:
if extract_new:
    for cesmd_zip in glob.glob(str(building_dir)+"/*.zip"):
        newdir = cesmd_zip[:-4]
        with ZipFile(cesmd_zip) as zObject: 
            zObject.extractall(path="./"+newdir)

In [8]:
move_new = False

In [9]:
if move_new:
    import os

    def delete_empty_folders(root):
        deleted = set()
        for current_dir, subdirs, files in os.walk(root, topdown=False):
            still_has_subdirs = False
            for subdir in subdirs:
                if os.path.join(current_dir, subdir) not in deleted:
                    still_has_subdirs = True
                    break
            if not any(files) and not still_has_subdirs:
                os.rmdir(current_dir)
                deleted.add(current_dir)
        return deleted

    for cesmd in BUILDINGS.keys():
        for record in glob.glob(str(building_dir)+"/"+cesmd+"/*/*/*.zip"):
            os.replace(record, str(building_dir)+"/"+cesmd+"/"+os.path.basename(record))
    
    delete_empty_folders(".")
    

In [10]:
summarize_new = True

In [11]:
if summarize_new:
    import quakeio
    import json
    for cesmd in BUILDINGS.keys():  
        events = []
        for file in glob.glob(str(building_dir)+"/"+cesmd+"/*.zip"):
            print(f"Reading {file}")
            try:
                events.append(
                    quakeio.read(file, summarize=True).serialize(serialize_data=False)
                )
            except:
                print(f"failed to read {file}")
        with open(str(building_dir)+"/"+cesmd+"_meta.json", "w") as writefile:
            json.dump(events, writefile)

Reading buildings/CE24602/BigBear92_ce24602p.zip
Reading buildings/CE24602/chinohills_29jul2008_ce24602p.zip
Reading buildings/CE24602/ci38443183_ce24602p.zip
Reading buildings/CE24602/ci38457511_ce24602p.zip
Reading buildings/CE24602/ci38695658_ce24602p.zip
Reading buildings/CE24602/ci39838928_ce24602p.zip
Reading buildings/CE24602/encino_17mar2014_15476961_ce24602p.zip
Reading buildings/CE24602/lahabra_28mar2014_15481673_ce24602p.zip
Reading buildings/CE24602/landers92_ce24602p.zip
Reading buildings/CE24602/northridge_17jan1994_ce24602p.zip
Reading buildings/CE24602/SierraMadre91_ce24602p.zip


In [12]:
building_names = [{'cesmd': cesmd, 'name': name} for cesmd,name in BUILDINGS.items()]

In [13]:
with open("building_names.json", "w") as writefile:
    json.dump(building_names, writefile)

In [14]:
# Don't worry about this; it's for making webpages of plots of building accelerations

for cesmd in BUILDINGS.keys():  
    print(f"""
    <div id='{cesmd}_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='{cesmd}_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>""", end="")


    <div id='CE24602_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='CE24602_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>